In [1]:
import numpy as np
import pandas as pd
import glob
import time
import math

In [2]:
DataPath1 = "I:/DataSets/CIBR_data/FL/*"
DataPath2 = "I:/DataSets/CIBR_data/insight/*"
savePath = "I:/DataSets/CIBR_data/Slices_FL/"
fs = 500
duration = 30
stride = 3
sleep_dict = {"W":0, "N1": 0, "N2": 1, "N3": 2, "R": 3}

duration *= fs
stride *= fs

In [3]:
def channel_pick(channelPath):
    channel = np.load(channelPath)
    print(channel)
    EEG_Pick = 'C3-M2'
    EOG_Pick = 'REOG'
    EMG_Pick = 'LEMG'
    
    EEG_Pick = int(np.where(channel==EEG_Pick)[0])
    EOG_Pick = int(np.where(channel==EOG_Pick)[0])
    EMG_Pick = int(np.where(channel==EMG_Pick)[0])
    
    print("Pick Channel:", channel[EEG_Pick], channel[EOG_Pick], channel[EMG_Pick])
    return EEG_Pick, EOG_Pick, EMG_Pick

In [4]:
def DC_remove(data):
    data = data - np.mean(data)
    return data

In [5]:
def stage_Cal(label):
    count_W_N1 = label.count(0)
    count_N2 = label.count(1)
    count_N3 = label.count(2)
    count_REM = label.count(3)
    
    stage_distribute = np.array([count_W_N1, count_N2, count_N3, count_REM])
    stage_max = max(stage_distribute)
    stage_index = np.where(stage_distribute == stage_max)[0]
    
    result = 0
    
    if 0 in stage_index:
        result = 0
    elif 3 in stage_index:
        result = 3
    elif 2 in stage_index:
        result = 2
    elif 1 in stage_index:
        result = 1
    
    return result

In [6]:
subjects_FL = glob.glob(DataPath1)
FL_DataPath = []
FL_Label = []
FL_Channel = []
index = []
start_from = 1

In [7]:
for name in subjects_FL:
    indx = name[-3:]
    index.append(indx)
    FL_DataPath.append(name + "/%s_eeg_data.npy" % indx)
    FL_Label.append(name + "/hypno_30s.csv")
    FL_Channel.append(name + "/%s_eeg_channel.npy" % indx)

In [8]:
for name in range(len(FL_DataPath)):
    if name < start_from:
        continue
    time0 = time.time()
    data = np.load(FL_DataPath[name])
    label_total = pd.read_csv(FL_Label[name])
    onsets = list(label_total["onset"])
    description = list(label_total["description"])
    
    label = []
    
    for i in range(len(description)):
        stage_temp = sleep_dict[description[i]]
        for _ in range(duration):
            label.append(stage_temp)
    total_len = len(label)
    
    EEG_Pick, EOG_Pick, EMG_Pick = channel_pick(FL_Channel[name])
    
    
    data_C3 = data[EEG_Pick][:total_len]
    data_REOG = data[EOG_Pick][:total_len]
    data_EMG = data[EMG_Pick][:total_len]
    
    data_C3 = DC_remove(data_C3)
    data_REOG = DC_remove(data_REOG)
    data_EMG = DC_remove(data_EMG)
    
    data = np.array([data_C3, data_REOG, data_EMG])
    
    num_windows = math.floor((total_len - duration) / stride) + 1
    print("Number of window:", num_windows)
    
    start = 0
    i = 0

    while start + duration <= total_len:
        end = start + duration
        data_temp = data[:, start:end]
        label_window = label[start:end]
        label_window = stage_Cal(label_window)
        saveName = savePath + "%s-%04d-stage%d.npy" % (index[name], i + 1, label_window)
        print(saveName)
        # np.save(saveName, data_temp)
        start += stride
        i += 1
    
    break

['F3-M2' 'F4-M1' 'C3-M2' 'C4-M1' 'O1-M2' 'O2-M1' 'REOG' 'LEMG' 'REMG'
 'LEOG']
Pick Channel: C3-M2 REOG LEMG
Number of window: 7461
I:/DataSets/CIBR_data/Slices_FL/006-0001-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0002-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0003-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0004-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0005-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0006-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0007-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0008-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0009-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0010-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0011-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0012-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0013-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0014-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0015-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-0016-stage0.npy
I:/DataSets/CIBR_data/Slices_FL/006-

In [9]:
for name in range(len(FL_DataPath)):
    time0 = time.time()
    label_total = pd.read_csv(FL_Label[name])
    onsets = list(label_total["onset"])
    for i in range(len(onsets)):
        onsets[i] = int(onsets[i] * fs)
    description = list(label_total["description"])
    for i in range(len(description)):
        description[i] = sleep_dict[description[i]]

    data = np.load(FL_DataPath[name])
    shape = data.shape
    count = len(description)
    print(count)

    for i in range(count):
        data_temp = data[:, onsets[i]:onsets[i] + duration]
        saveName = savePath + "%s-%04d-stage%d.npy" % (index[name], i+1, description[i])
        np.save(saveName, data_temp)
    
    print("Done! Use Time: %.3f" % (time.time() - time0))
        